In [1]:
import utils
import collaborative_filtering as cf

In [2]:
Y_train = utils.get_training_data()
Y_test = utils.get_test_data()

M = 943 # users
N = 1682 # movies
K = 10
reg = 0.1
eta = 0.03

In [3]:
U, V, err = cf.train_model(Y_train, M, N, K, eta, reg)
err_test = cf.get_err(U, V, Y_test)

Epoch  0  of  300
Epoch  10  of  300


In [8]:
print('Average training error =', '{:.3f}'.format(err/len(Y_train)))
print('Average test error =', '{:.3f}'.format(err_test/len(Y_test)))

Average training error = 0.350
Average test error = 0.455
